# Plane Waves

In developing the techniques to deal with the periodicity of a crystalline lattice, we found that the wavefunction is periodic up to a phase given as

\begin{equation}
\psi(\mathbf{r}+\mathbf{R_n}) = e^{i\mathbf{k}\cdot\mathbf{R_n}}\psi(\mathbf{r}).
\end{equation}

Using the fact that the wavefunction is periodic in real space, it can be written as a Fourier series in momenum space as:

\begin{equation}
\psi(\mathbf{r}) = \sum_\mathbf{k} C_\mathbf{k} e^{i\mathbf{k}\cdot\mathbf{r}}
\end{equation}

where $C_\mathbf{k}$ is an expansion coefficent. Recall that we used this to treat a free particle confined to a lattice. This procedure can be extended to include a pair potential that describes the interaction between particles in the lattice. Since the potential must be periodic it can also be written as a Fourier series as

\begin{equation}
\hat{V}(\mathbf{r}) = \sum_{\mathbf{G_n}} V_{\mathbf{G_n}} e^{i \mathbf{G_n} \cdot \mathbf{r}}
\end{equation}

Using these in Schrodinger's equation results in another method to calculate the energy spectrum.

\begin{equation}
\frac{-\hbar^2}{2m} \sum_{\mathbf{k}} \nabla^2 C_\mathbf{k} e^{i \mathbf{k} \cdot \mathbf{r}} + \left( \sum_{\mathbf{G_n}} V_{\mathbf{G_n}} e^{i \mathbf{G_n} \cdot \mathbf{r}} \right) \left(\sum_{\mathbf{k}} C_\mathbf{k} e^{i \mathbf{k} \cdot \mathbf{r}} \right) = E \sum_{\mathbf{k}} C_\mathbf{k} e^{i \mathbf{k} \cdot \mathbf{r}},
\end{equation}

This simplifies to

\begin{equation}
\sum_{\mathbf{k}} e^{i \mathbf{k} \cdot \mathbf{r}} \left[ \left( \frac{\hbar^2 k^2}{2m} - E \right) C_{\mathbf{k}} + \sum_{\mathbf{G_n}} V_{\mathbf{G_n}} C_{\mathbf{k} - \mathbf{G_n}} \right] = 0
\end{equation}

Therefore, for each $\mathbf{k}$ the energy can be calculated from the central equation:

\begin{equation}
\left( \frac{\hbar^2 ( \mathbf{k} - \mathbf{G_n})^2}{2m} - E \right) C_{\mathbf{k} - \mathbf{G_n}} + \sum_{\mathbf{G_n}} V_{\mathbf{G_n}} C_{\mathbf{k} - \mathbf{G_n} - \mathbf{G_n}'} = 0
\end{equation}

When ${V(\mathbf{r})} = 0$, the result obtain from the empty lattice approximation is recovered. Now consider when

\begin{equation}
V(\mathbf{r}) = \sum_{\mathbf{R_n}}\frac{Ze^2}{|\mathbf{r}-\mathbf{Rn}|} = \sum_{\mathbf{G_n}} \frac{e^2Ze^{i \mathbf{G_n} \cdot (\mathbf{r} - \mathbf{R_n})}}{(\mathbf{G_n} - \mathbf{G_n}')^2}
\end{equation}

In [1]:
import numpy as np
import seekpath
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from ipywidgets import interact, FloatSlider
import cmath

def solver(Z,a):
    plt.rcParams.update({'font.size': 14})
    fig, ax = plt.subplots(figsize=(5,5))
    
    ax.tick_params(axis='both',direction='in')
    plt.ylabel('Energy')
    
    structure = ([[a,0,0],[0,a,0],[0,0,a]],[[0,0,0]],[1])
    k_path = seekpath.get_explicit_k_path(structure=structure)['explicit_kpoints_abs']
    k_ticks = seekpath.get_explicit_k_path(structure=structure)['explicit_kpoints_linearcoord']
    k_labels = seekpath.get_explicit_k_path(structure=structure)['explicit_kpoints_labels']
    k_segments = seekpath.get_explicit_k_path(structure=structure)['explicit_segments']
    hamiltonian = np.arange(9*len(k_ticks),dtype=complex).reshape(3,3,len(k_ticks))
    energy = np.arange(3*len(k_ticks),dtype=complex).reshape(3,len(k_ticks))
    eig_vects = np.arange(9*len(k_ticks),dtype=complex).reshape(3,3,len(k_ticks))
    
    lat = structure[0]
    position = structure[1]
    
    G = np.zeros((3, 3))
    G[0] = 2* np.pi * np.cross(lat[1], lat[2]) / (np.inner(
            lat[0], np.cross(lat[1], lat[2])))
    G[1] = 2* np.pi * np.cross(lat[2], lat[0]) / (np.inner(
            lat[1], np.cross(lat[2], lat[0])))
    G[2] =  2* np.pi * np.cross(lat[0], lat[1]) / (np.inner(
            lat[2], np.cross(lat[0], lat[1])))
    
    kpath = np.inner(G,k_path/(2*np.pi)).T
    
    for k in range(len(kpath)):
        for iham in range(0,3):
            for jham in range(0,3):
                if iham == jham:
                    hamiltonian[iham][jham][k] = np.inner((kpath[k] - G[iham]),(kpath[k] - G[iham]))/2
                else:
                    hamiltonian[iham][jham][k] = -Z/np.inner((G[iham] - G[jham]),(G[iham] - G[jham]))
        
        hamiltonian[:,:,k] = 1/2*(hamiltonian[:,:,k] + hamiltonian[:,:,k].conj().T)

        energy[:,k], eig_vects[:,:,k] = np.linalg.eigh(hamiltonian[:,:,k])[:]

    
    for i in range(3):
        plt.plot(k_ticks,energy[i,:].real)
    
    plt.xlim(0,max(k_ticks))
    
    for i in range(len(k_segments[:])):
        if k_segments[i-1][1] == k_segments[i][0]:
            k_labels[k_segments[i-1][1]] = k_labels[k_segments[i][0]-1] + '|' + k_labels[k_segments[i][0]]
            k_labels[k_segments[i][0]-1] = ''
            plt.axvline(x=k_ticks[k_segments[i][0]],color='black')
        else:
            pass
    
    for i in range(len(k_labels)):
        if k_labels[i] == "GAMMA":
            k_labels[i] = "\Gamma"
        else:
            pass

    k_labels = [str("$" + latx + "$") for latx in k_labels]
    
    plt.xticks([k_ticks[i] for i,x in enumerate(k_labels) if x != '$$'],[k_labels[i] for i,x in enumerate(k_labels) if x != '$$'])
    
    plt.show()

interactive_plot = interact(solver, Z=FloatSlider(value=0, min=0, max=100, step=1, description='$Z$:'),
                           a=FloatSlider(value=1, min=0.5, max=4, step=0.5, description='$a$:'))
interactive_plot

interactive(children=(FloatSlider(value=0.0, description='$Z$:', step=1.0), FloatSlider(value=1.0, description…

<function __main__.solver(Z, a)>

In [2]:
import numpy as np
import seekpath
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from ipywidgets import interact, FloatSlider
import cmath

def solver(Z,a):
    plt.rcParams.update({'font.size': 14})
    fig, ax = plt.subplots(figsize=(5,5))
    
    ax.tick_params(axis='both',direction='in')
    plt.ylabel('Energy')
    
    structure = ([[-a/2,a/2,a/2],[a/2,-a/2,a/2],[a/2,a/2,-a/2]],[[0,0,0]],[1])
    k_path = seekpath.get_explicit_k_path(structure=structure)['explicit_kpoints_abs']
    k_ticks = seekpath.get_explicit_k_path(structure=structure)['explicit_kpoints_linearcoord']
    k_labels = seekpath.get_explicit_k_path(structure=structure)['explicit_kpoints_labels']
    k_segments = seekpath.get_explicit_k_path(structure=structure)['explicit_segments']
    hamiltonian = np.arange(9*len(k_ticks),dtype=complex).reshape(3,3,len(k_ticks))
    energy = np.arange(3*len(k_ticks),dtype=complex).reshape(3,len(k_ticks))
    eig_vects = np.arange(9*len(k_ticks),dtype=complex).reshape(3,3,len(k_ticks))
    
    lat = structure[0]
    position = structure[1]
    
    G = np.zeros((3, 3))
    G[0] = 2* np.pi * np.cross(lat[1], lat[2]) / (np.inner(
            lat[0], np.cross(lat[1], lat[2])))
    G[1] = 2* np.pi * np.cross(lat[2], lat[0]) / (np.inner(
            lat[1], np.cross(lat[2], lat[0])))
    G[2] =  2* np.pi * np.cross(lat[0], lat[1]) / (np.inner(
            lat[2], np.cross(lat[0], lat[1])))
    
    kpath = np.inner(G,k_path/(2*np.pi)).T
    
    for k in range(len(kpath)):
        for iham in range(0,3):
            for jham in range(0,3):
                if iham == jham:
                    hamiltonian[iham][jham][k] = np.inner((kpath[k] - G[iham]),(kpath[k] - G[iham]))/2
                else:
                    hamiltonian[iham][jham][k] = -Z/np.inner((G[iham] - G[jham]),(G[iham] - G[jham]))
        
        hamiltonian[:,:,k] = 1/2*(hamiltonian[:,:,k] + hamiltonian[:,:,k].conj().T)

        energy[:,k], eig_vects[:,:,k] = np.linalg.eigh(hamiltonian[:,:,k])[:]

    
    for i in range(3):
        plt.plot(k_ticks,energy[i,:].real)
    
    plt.xlim(0,max(k_ticks))
    
    for i in range(len(k_segments[:])):
        if k_segments[i-1][1] == k_segments[i][0]:
            k_labels[k_segments[i-1][1]] = k_labels[k_segments[i][0]-1] + '|' + k_labels[k_segments[i][0]]
            k_labels[k_segments[i][0]-1] = ''
            plt.axvline(x=k_ticks[k_segments[i][0]],color='black')
        else:
            pass
    
    for i in range(len(k_labels)):
        if k_labels[i] == "GAMMA":
            k_labels[i] = "\Gamma"
        else:
            pass

    k_labels = [str("$" + latx + "$") for latx in k_labels]
    
    plt.xticks([k_ticks[i] for i,x in enumerate(k_labels) if x != '$$'],[k_labels[i] for i,x in enumerate(k_labels) if x != '$$'])
    
    plt.show()

interactive_plot = interact(solver, Z=FloatSlider(value=0, min=0, max=100, step=1, description='$Z$:'),
                           a=FloatSlider(value=1, min=0.5, max=4, step=0.1, description='$a$:'))
interactive_plot

interactive(children=(FloatSlider(value=0.0, description='$Z$:', step=1.0), FloatSlider(value=1.0, description…

<function __main__.solver(Z, a)>

In [4]:
import numpy as np
import seekpath
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from ipywidgets import interact, FloatSlider
import cmath

def solver(Z,a):
    plt.rcParams.update({'font.size': 14})
    fig, ax = plt.subplots(figsize=(5,5))
    
    ax.tick_params(axis='both',direction='in')
    plt.ylabel('Energy')
    
    structure = ([[a/2,a/2,0],[a/2,0,a/2],[0,a/2,a/2]],[[0,0,0]],[1])
    k_path = seekpath.get_explicit_k_path(structure=structure)['explicit_kpoints_abs']
    k_ticks = seekpath.get_explicit_k_path(structure=structure)['explicit_kpoints_linearcoord']
    k_labels = seekpath.get_explicit_k_path(structure=structure)['explicit_kpoints_labels']
    k_segments = seekpath.get_explicit_k_path(structure=structure)['explicit_segments']
    hamiltonian = np.arange(9*len(k_ticks),dtype=complex).reshape(3,3,len(k_ticks))
    energy = np.arange(3*len(k_ticks),dtype=complex).reshape(3,len(k_ticks))
    eig_vects = np.arange(9*len(k_ticks),dtype=complex).reshape(3,3,len(k_ticks))
    
    lat = structure[0]
    position = structure[1]
    
    G = np.zeros((3, 3))
    G[0] = 2* np.pi * np.cross(lat[1], lat[2]) / (np.inner(
            lat[0], np.cross(lat[1], lat[2])))
    G[1] = 2* np.pi * np.cross(lat[2], lat[0]) / (np.inner(
            lat[1], np.cross(lat[2], lat[0])))
    G[2] =  2* np.pi * np.cross(lat[0], lat[1]) / (np.inner(
            lat[2], np.cross(lat[0], lat[1])))
    
    kpath = np.inner(G,k_path/(2*np.pi)).T
    
    for k in range(len(kpath)):
        for iham in range(0,3):
            for jham in range(0,3):
                if iham == jham:
                    hamiltonian[iham][jham][k] = np.inner((kpath[k] - G[iham]),(kpath[k] - G[iham]))/2
                else:
                    hamiltonian[iham][jham][k] = -Z/np.inner((G[iham] - G[jham]),(G[iham] - G[jham]))
        
        hamiltonian[:,:,k] = 1/2*(hamiltonian[:,:,k] + hamiltonian[:,:,k].conj().T)

        energy[:,k], eig_vects[:,:,k] = np.linalg.eigh(hamiltonian[:,:,k])[:]

    
    for i in range(3):
        plt.plot(k_ticks,energy[i,:].real)
    
    plt.xlim(0,max(k_ticks))
    
    for i in range(len(k_segments[:])):
        if k_segments[i-1][1] == k_segments[i][0]:
            k_labels[k_segments[i-1][1]] = k_labels[k_segments[i][0]-1] + '|' + k_labels[k_segments[i][0]]
            k_labels[k_segments[i][0]-1] = ''
            plt.axvline(x=k_ticks[k_segments[i][0]],color='black')
        else:
            pass
    
    for i in range(len(k_labels)):
        if k_labels[i] == "GAMMA":
            k_labels[i] = "\Gamma"
        else:
            pass

    k_labels = [str("$" + latx + "$") for latx in k_labels]
    
    plt.xticks([k_ticks[i] for i,x in enumerate(k_labels) if x != '$$'],[k_labels[i] for i,x in enumerate(k_labels) if x != '$$'])
    
    plt.show()

interactive_plot = interact(solver, Z=FloatSlider(value=0, min=0, max=100, step=1, description='$Z$:'),
                           a=FloatSlider(value=1, min=0.5, max=4, step=0.1, description='$a$:'))
interactive_plot

interactive(children=(FloatSlider(value=0.0, description='$Z$:', step=1.0), FloatSlider(value=1.0, description…

<function __main__.solver(Z, a)>

In [3]:
import numpy as np
import seekpath
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from ipywidgets import interact, FloatSlider
import cmath

def solver(Z,a,c):
    plt.rcParams.update({'font.size': 14})
    fig, ax = plt.subplots(figsize=(5,5))
    
    ax.tick_params(axis='both',direction='in')
    plt.ylabel('Energy')
    
    structure = ([[a/2,np.sqrt(3)*a/2,0],[a/2,-np.sqrt(3)*a/2,0],[0,0,c]],[[0,0,0]],[1])
    k_path = seekpath.get_explicit_k_path(structure=structure)['explicit_kpoints_abs']
    k_ticks = seekpath.get_explicit_k_path(structure=structure)['explicit_kpoints_linearcoord']
    k_labels = seekpath.get_explicit_k_path(structure=structure)['explicit_kpoints_labels']
    k_segments = seekpath.get_explicit_k_path(structure=structure)['explicit_segments']
    hamiltonian = np.arange(9*len(k_ticks),dtype=complex).reshape(3,3,len(k_ticks))
    energy = np.arange(3*len(k_ticks),dtype=complex).reshape(3,len(k_ticks))
    eig_vects = np.arange(9*len(k_ticks),dtype=complex).reshape(3,3,len(k_ticks))
    
    lat = structure[0]
    position = structure[1]
    
    G = np.zeros((3, 3))
    G[0] = 2* np.pi * np.cross(lat[1], lat[2]) / (np.inner(
            lat[0], np.cross(lat[1], lat[2])))
    G[1] = 2* np.pi * np.cross(lat[2], lat[0]) / (np.inner(
            lat[1], np.cross(lat[2], lat[0])))
    G[2] =  2* np.pi * np.cross(lat[0], lat[1]) / (np.inner(
            lat[2], np.cross(lat[0], lat[1])))
    
    kpath = np.inner(G,k_path/(2*np.pi)).T
    
    for k in range(len(kpath)):
        for iham in range(0,3):
            for jham in range(0,3):
                if iham == jham:
                    hamiltonian[iham][jham][k] = np.inner((kpath[k] - G[iham]),(kpath[k] - G[iham]))/2
                else:
                    hamiltonian[iham][jham][k] = -Z/np.inner((G[iham] - G[jham]),(G[iham] - G[jham]))
        
        hamiltonian[:,:,k] = 1/2*(hamiltonian[:,:,k] + hamiltonian[:,:,k].conj().T)

        energy[:,k], eig_vects[:,:,k] = np.linalg.eigh(hamiltonian[:,:,k])[:]

    
    for i in range(3):
        plt.plot(k_ticks,energy[i,:].real)
    
    plt.xlim(0,max(k_ticks))
    
    for i in range(len(k_segments[:])):
        if k_segments[i-1][1] == k_segments[i][0]:
            k_labels[k_segments[i-1][1]] = k_labels[k_segments[i][0]-1] + '|' + k_labels[k_segments[i][0]]
            k_labels[k_segments[i][0]-1] = ''
            plt.axvline(x=k_ticks[k_segments[i][0]],color='black')
        else:
            pass
    
    for i in range(len(k_labels)):
        if k_labels[i] == "GAMMA":
            k_labels[i] = "\Gamma"
        else:
            pass

    k_labels = [str("$" + latx + "$") for latx in k_labels]
    
    plt.xticks([k_ticks[i] for i,x in enumerate(k_labels) if x != '$$'],[k_labels[i] for i,x in enumerate(k_labels) if x != '$$'])
    
    plt.show()

interactive_plot = interact(solver, Z=FloatSlider(value=0, min=0, max=100, step=1, description='$Z$:'),
                           a=FloatSlider(value=1, min=0.5, max=4, step=0.1, description='$a$:'),
                           c=FloatSlider(value=1, min=0.5, max=4, step=0.1, description='$c$:'))
interactive_plot

interactive(children=(FloatSlider(value=0.0, description='$Z$:', step=1.0), FloatSlider(value=1.0, description…

<function __main__.solver(Z, a, c)>